# Install the required packages

In [ ]:
import sys
import os
import glob
import matplotlib.pyplot as plt

import numpy as np
#to work with text files
import pandas as pd 

#pre-processing imports
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords') # stopword 
nltk.download('punkt') # word_tokenize

#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# word2vec installation
- There is no need to change the video texts to lower case as there are some words are capital letters in **word2vec**

In [ ]:
!brew install wget
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
data_path= "DATAPATH"
%time 
w2v_model = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)
print('done loading Word2Vec')

/bin/bash: brew: command not found
--2021-12-01 21:14:15--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.88.230
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.88.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  65.0MB/s    in 24s     

2021-12-01 21:14:39 (65.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs
done loading Word2Vec


In [ ]:
# Inspect the model
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]
word2vec_vocab_show = [item for item in word2vec_vocab]
print(len(word2vec_vocab))
print(word2vec_vocab_show[500:1000])

3000000
['doing', 'face', 'low', 'higher', 'site', 'once', 'yet', 'hours', 'America', 'control', 'received', 'rate', 'career', 'Bush', 'teams', 'known', 'offer', 'race', 'ever', 'experience', 'playing', 'name', 'possible', 'countries', 'Mr.', 'average', 'together', 'using', '9', 'cut', 'While', 'total', 'round', 'young', 'nearly', 'shares', 'member', 'campaign', 'media', 'needs', 'why', 'house', 'issues', 'costs', 'fire', '##-#', 'victory', 'player', 'began', 'sure', 'story', 'per_cent', 'North', 'His', 'staff', 'order', 'war', 'large', 'interest', 'stock', 'food', 'research', 'key', 'India', 'South', 'morning', 'conference', 'senior', 'global', 'Center', 'death', 'person', 'thought', 'gave', 'feel', 'energy', 'history', 'recently', 'largest', 'No.', 'general', 'official', 'released', 'wanted', 'meet', 'short', 'outside', 'running', 'live', 'ball', 'online', 'real', 'position', 'fact', 'fell', 'nine', 'December', 'front', 'action', 'defense', 'problem', 'problems', 'Mr', 'nation', 'nee

In [ ]:
# Ref: https://www.geeksforgeeks.org/python-os-chdir-method/
# Ref: https://stackoverflow.com/questions/9234560/find-all-csv-files-in-a-directory-using-python/12280052 

def get_list_filenames(cwd, path, extension):
    try:
        os.chdir(path)
        print("Successfully change the directory!")
        filenames = glob.glob('*.{}'.format(extension))
        print("Number of files: ", len(filenames))

    # Caching the exception
    except: 
        print("Something wrong with specified directory. Exception- ", sys.exc_info()) 

    # handling with finally: restore the path which is the current directory before changing directory           
    finally: 
        print("Restoring the path") 
        os.chdir(cwd) # Change it back to cwd (original directory)
        print("Current directory is-", os.getcwd()) 
        
    return filenames


# Ref: https://realpython.com/python-keras-text-classification/

# Get a list of dataframe after reading all csv files given a path
def get_all_image_files_read(path, filenames):
    list_img = []
    for file in filenames:
        img = imageio.imread(path+file)
        list_img.append(img)
    return list_img

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
!pwd

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake


In [ ]:
# initial directory 
cwd = os.getcwd()
path_org = './' # the directory where all the output result are located.
extension = 'txt'

fake_textfiles = get_list_filenames(cwd, path_org, extension)
print(fake_textfiles[0])
print(len(fake_textfiles))

Successfully change the directory!
Number of files:  4500
Restoring the path
Current directory is- /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
FAKE_BCD3501.txt
4500


- 360 from B fake videos
- 1884 from C fake videos
- 2256 from D fake videos

Totally 4500 text files

In [ ]:
print(fake_textfiles[2300])

FAKE_BCD1801.txt


# Tokenize, Remove Stopwords, and Lowercase Conversion
- Keeping punctuations could be relevant for this task

In [ ]:
aaa = []
aaa.append(open(fake_textfiles[786]).readline())
print(aaa)

mystopwords = set(stopwords.words("english"))
# function to tokenize tweets, remove stopwords and numbers.
#k eeping punctuations and emoticon symbols could be relevant for this task!
def preprocess_corpus(texts):
  def remove_stops_digits(tokens):
  #Nested function that removes stopwords and digits from a list of tokens
    return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    #This return statement below uses the above function to process twitter tokenizer output further.
  return [remove_stops_digits(word_tokenize(content.lower())) for content in texts]


bbb = preprocess_corpus(aaa)
flatten_list = sum(bbb, [])
print(flatten_list)


['the walls that every time they play every note has the same amount of weight and import']
['walls', 'every', 'time', 'play', 'every', 'note', 'amount', 'weight', 'import']


# Create **word2vec** Vetorzation Function

In [ ]:
# Creating a feature vector by averaging all embeddings for all sentences
def embedding_feats(list_of_lists):
  DIMENSION = 300
  zero_vector = np.zeros(DIMENSION)
  feats = []
  for tokens in list_of_lists:
    feat_for_this = np.zeros(DIMENSION)
    count_for_this = 0
    for token in tokens:
      if token in w2v_model:
        feat_for_this += w2v_model[token]
        count_for_this +=1
    feats.append(feat_for_this/count_for_this)
  feats = np.mean(feats, axis = 0)
  return feats


train_vector = embedding_feats(flatten_list)
train_vector = np.array(train_vector)
#train_vector = np.mean(train_vector, axis = 0)
print(train_vector.shape)
print(train_vector)

(300,)
[-0.1509769   0.12244195 -0.03106079  0.13832827 -0.03958898  0.05941467
 -0.07734251 -0.01856283 -0.05395621  0.03449888 -0.06506218 -0.09933958
 -0.21510756  0.01443549 -0.12373861  0.11568332  0.12866505  0.2039804
 -0.00758707  0.0004516  -0.29005986 -0.06603351  0.12096942  0.04745466
 -0.08604013  0.00758191 -0.26892135  0.06359773 -0.05426071 -0.06032138
  0.00208415  0.01516861 -0.08044581 -0.11542811 -0.13814629  0.07139278
 -0.22603986  0.12177147 -0.04786462  0.07946868 -0.05105591 -0.06170654
  0.03582628  0.1057529   0.06138496 -0.05319413 -0.06387612 -0.19568866
 -0.12626524  0.09331982 -0.17814015  0.27457049 -0.01637189  0.20968153
  0.06768889  0.14569137 -0.18095613 -0.07193134  0.00971657 -0.18875823
 -0.15778662 -0.08794047 -0.19061641 -0.06888586 -0.05242377 -0.19155183
 -0.10826009  0.09541129 -0.04131402  0.08348411  0.02170003 -0.0531042
  0.07875683  0.02169201 -0.0487036  -0.00353913  0.17202363  0.02102966
 -0.02220278 -0.09103913 -0.12676957 -0.009602

# create `FAKE_RTVC_B` text features
- total frames: 360 videos * 4 frames = 1440

In [ ]:
fake_B = []
for i in range(0, 360):
  openlist = []
  openlist.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(openlist)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=np.float32)
  #final = np.ndarray.flatten(text_vectors)
  final = text_vectors
  print(final.shape)
  #del tmp
  fake_B.append(final)

(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)


In [ ]:
print(len(fake_B))
#print(fake_B[100])

# Because we capture 4 frame in fake B set, so we need to duplicate the text 4 times 
B = np.concatenate((fake_B, fake_B, fake_B, fake_B), axis = 0)
print(len(B))

360
1440


In [ ]:
B = np.array(B)
print(B.shape)

(1440, 300)


# Create `FAKE_FSGAN_C` to text features
- total frames: 1884 videos * 2 frames = 3768

In [ ]:
fake_C = []
for i in range(360, 2244):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=np.float32)
  final = text_vectors
  print(final.shape)
  del tmp
  fake_C.append(final)

(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)

In [ ]:
print(len(fake_C))
#print(fake_B[100])

# Because we capture 2 frame in fake C set , so we need to duplicate the text 2 times 
C = np.concatenate((fake_C, fake_C), axis = 0)
print(len(C))

1884
3768


# Create `FAKE_FACESWAP_D_W2L` to text features
- total frames: 2256 videos * 2 frames = 4512

In [ ]:
fake_D = []
for i in range(2244, len(fake_textfiles)):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=np.float32)
  #final = np.ndarray.flatten(text_vectors)
  final = text_vectors
  print(final.shape)
  del tmp
  fake_D.append(final)

(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)

In [ ]:
print(len(fake_D))
#print(fake_B[100])

# Because we capture 2 frame in fake D set , so we need to duplicate the text 2 times 
D = np.concatenate((fake_D, fake_D), axis = 0)
print(len(D))

2256
4512


# Save BCD  as fake `npy file`
- The number of fake text observations is 9720 

In [ ]:
B, C, D = np.array(B), np.array(C), np.array(D)
fake_texts = np.concatenate((B, C, D), axis = 0)

print(len(fake_texts))
print(fake_texts[1000].shape)
print(fake_texts[2000])
print(fake_texts[1000])

9720
(300,)
[-1.80964500e-01  1.32426962e-01 -3.02937087e-02  1.38394639e-01
 -4.24308293e-02  1.97700057e-02 -8.86552408e-02 -2.57581770e-02
 -3.47200744e-02  2.31732186e-02 -5.99324182e-02 -5.91804013e-02
 -1.89397812e-01  2.96444632e-02 -9.45358425e-02  1.11077964e-01
  9.49481130e-02  1.74661949e-01 -3.60787772e-02  2.14050729e-02
 -2.71214545e-01 -3.63294445e-02  1.02535449e-01 -9.29308706e-04
 -7.10219294e-02  1.16040837e-02 -2.52752215e-01  6.83756247e-02
 -4.67496589e-02 -3.90845761e-02  1.46350043e-03  2.85240542e-02
 -6.63555339e-02 -8.20412114e-02 -1.19263232e-01  1.01348415e-01
 -2.48531014e-01  1.10863417e-01 -2.67512426e-02  7.62973130e-02
 -4.30956446e-02 -5.15636019e-02  6.81697130e-02  1.01160862e-01
  4.73957993e-02 -2.60570310e-02 -5.02982810e-02 -1.58562034e-01
 -1.18063591e-01  9.90657732e-02 -1.46812364e-01  2.33555779e-01
 -3.87747698e-02  2.51694947e-01  8.50453302e-02  1.27667680e-01
 -1.48713812e-01 -9.47464034e-02 -1.85325909e-02 -1.61480799e-01
 -1.39585614e

In [ ]:
print(fake_texts[300])

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
!pwd
np.save("fake_texts.npy", fake_texts)

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data


# Create `REAL_A` text features

- total **REAL** images: 390 videos * 25 frames = 9750

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real
!pwd

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real


In [ ]:
# initial directory 
cwd = os.getcwd()
path_org = './' # the directory where all the output result are located.
extension = 'txt'

real_textfiles = get_list_filenames(cwd, path_org, extension)
print(real_textfiles[0])
print(len(real_textfiles))

Successfully change the directory!
Number of files:  390
Restoring the path
Current directory is- /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real
REAL_A0.txt
390


In [ ]:
real_A = []
for i in range(len(real_textfiles)):
  tmp = []
  tmp.append(open(real_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=np.float32)
  print(text_vectors.shape)
  del tmp
  real_A.append(text_vectors)

(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)

In [ ]:
A = np.array(real_A, dtype=object)
real_texts = np.concatenate((A, A, A, A, A,
                             A, A, A, A, A,
                             A, A, A, A, A,
                             A, A, A, A, A,
                             A, A, A, A, A), axis = 0)
print(len(real_texts))
print(real_texts.shape)

9750
(9750, 300)


# Save A as real `npy file`
- The number of fake text observations is 9750

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
!pwd
np.save("real_texts.npy", real_texts)

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data


# Concatenate fake and real arrays then save as `npy` file

For $X_2$ text features, we have
- real video: 9750 observations from `Real_A`.
- fake videos: 9720 observations from `FAKE_RTVC_B`, `FAKE_FSGAN_C` and `FAKE_FACESWAP_D_W2L`.
- The number of observations is 19470

In [ ]:
X2_texts = np.concatenate((fake_texts, real_texts), axis = 0)
print(len(X2_texts))
print(X2_texts.shape)
np.save("X2_texts.npy", X2_texts)

19470
(19470, 300)


# References
- https://code.google.com/archive/p/word2vec/